In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# from keras.utils.np_utils import to_categorical
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('data/data.csv', sep=';')
df.head()

,year,team,standing
0,2015,Arsenal,3
1,2016,Arsenal,2
2,2017,Arsenal,5
3,2018,Arsenal,6
4,2019,Arsenal,5


In [3]:
vals = df.values
sequences = 2
teams = np.unique(df['team'].values)
print(teams)
X = []
y = []
X_test = []
X_test_teams = []


df_dummies = pd.get_dummies(df, columns=['team'])
df_filtered = df_dummies.drop(columns=['year'])

for team in teams:
    df_team = df_filtered.loc[df['team'] == team]
    #print(df_team)
    team_standings = df_team.values
    # print(f"len(team_standings) {len(team_standings)}")
    for i in range(0, len(team_standings)):
        # print(f"team {team} {i}th year")
        if i + sequences < len(team_standings):
            x_values = team_standings[i:i+sequences]
            #print(x_values)
            X.append(x_values)
            y_value = team_standings[i+sequences, 0]
            #print(y_value)
            y.append(y_value)
        elif i + sequences == len(team_standings):
            x_values = team_standings[i:i+sequences]
            X_test.append(x_values)
            print(f"team {team}")
            X_test_teams.append(team)

X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
print(X.shape)
print(y.shape)
print(X_test.shape)

['Arsenal' 'Aston Villa' 'Bournemouth' 'Brighton' 'Burnley' 'Cardiff City'
 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield' 'Hull City'
 'Leeds United' 'Leicester City' 'Liverpool' 'Man City' 'Man United'
 'Middlesbrough' 'Newcastle' 'Norwich City' 'Sheffield United'
 'Southampton' 'Stoke City' 'Sunderland' 'Swansea' 'Tottenham' 'Watford'
 'West Brom' 'West Ham' 'Wolves']
team Arsenal
team Aston Villa
team Bournemouth
team Brighton
team Burnley
team Cardiff City
team Chelsea
team Crystal Palace
team Everton
team Fulham
team Huddersfield
team Hull City
team Leeds United
team Leicester City
team Liverpool
team Man City
team Man United
team Newcastle
team Norwich City
team Sheffield United
team Southampton
team Stoke City
team Sunderland
team Swansea
team Tottenham
team Watford
team West Brom
team West Ham
team Wolves
(165, 2, 31)
(165,)
(29, 2, 31)


In [4]:
print(y)

[ 5  6  5  8  8  5 21 21 21 17 11 14  9 12 21 14 18 21 21 21 15 17 15 16
  9 16  7 15 10 17 18 18  1  5  3  4  4  3 14 11 12 14 14 12  7  8  8 12
 10 16 21 19 21 18 21 21 16 20 21 21 21 18 21 21 21 21 21 21 21 21 21  9
 17 12  9 21  9  5  5  8  4  4  2  1  3  2  3  1  1  2  1  1  6  2  6  3
  2  6 10 13 13 12 11 21 21 21 20 21 20 21 21 21  9 20 21  8 17 16 11 15
 15 13 19 21 21 21 21 21 20 21 21 21 21 21 15 18  2  3  4  6  7  4 14 11
 19 21 19 10 20 21 21 19 32 11 13 10 16  6  7 21 21  7  7 13 10]


In [8]:
print(X_test)

[[[ 8  1  0 ...  0  0  0]
  [ 5  1  0 ...  0  0  0]]

 [[11  0  1 ...  0  0  0]
  [14  0  1 ...  0  0  0]]

 [[21  0  0 ...  0  0  0]
  [21  0  0 ...  0  0  0]]

 ...

 [[19  0  0 ...  1  0  0]
  [32  0  0 ...  1  0  0]]

 [[ 6  0  0 ...  0  1  0]
  [ 7  0  0 ...  0  1  0]]

 [[13  0  0 ...  0  0  1]
  [10  0  0 ...  0  0  1]]]


In [7]:
def linear_transform(X):
    if len(X.shape) == 3:
        if X.shape[1] == 1:
            X_linear = np.squeeze(X, 1)
        else:
            X_linear = X.reshape((X.shape[0], X.shape[1]* X.shape[2]))
    return X_linear

In [8]:
X_linear = linear_transform(X)
print(X_linear.shape)

(165, 62)


In [9]:
print(X_linear.shape)
print(y.shape)
reg = LinearRegression().fit(X_linear, y)
reg.score(X_linear, y)

(165, 62)
(165,)


-2.8308396467534402

In [10]:
X_test_linear = linear_transform(X_test)
print(X_test_linear.shape)

(29, 62)


In [11]:
print(X_test_teams)

['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley', 'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield', 'Hull City', 'Leeds United', 'Leicester City', 'Liverpool', 'Man City', 'Man United', 'Newcastle', 'Norwich City', 'Sheffield United', 'Southampton', 'Stoke City', 'Sunderland', 'Swansea', 'Tottenham', 'Watford', 'West Brom', 'West Ham', 'Wolves']


In [12]:
result = reg.predict(X_test_linear)
print(np.array(result))

[ -2.29980469   6.99609375  17.87158203  -4.17236328  10.60644531
  17.56738281  18.76916504  -3.49609375  44.64892578  20.19238281
 -16.15576172  39.39550781  28.19238281   9.31738281  -3.78820801
  -4.29321289  -2.36914062  11.35229492  28.92285156  11.08691406
  14.56347656  20.52246094  43.63769531  17.71044922  -0.81640625
  17.74658203  21.80371094  13.47399902  23.72949219]


In [13]:
inds = np.argsort(result, axis=0)
print(inds)

[10 15  3 14  7 16  0 24  1 13  4 19 17 27 20  5 23 25  2  6  9 21 26 28
 12 18 11 22  8]


In [14]:
for i, ind in enumerate(inds):
    team = X_test_teams[ind]
    res = result[ind]
    print(f"{i+1}th team {team} is {round(res, 2)}")

1th team Huddersfield is -16.16
2th team Man City is -4.29
3th team Brighton is -4.17
4th team Liverpool is -3.79
5th team Crystal Palace is -3.5
6th team Man United is -2.37
7th team Arsenal is -2.3
8th team Tottenham is -0.82
9th team Aston Villa is 7.0
10th team Leicester City is 9.32
11th team Burnley is 10.61
12th team Sheffield United is 11.09
13th team Newcastle is 11.35
14th team West Ham is 13.47
15th team Southampton is 14.56
16th team Cardiff City is 17.57
17th team Swansea is 17.71
18th team Watford is 17.75
19th team Bournemouth is 17.87
20th team Chelsea is 18.77
21th team Fulham is 20.19
22th team Stoke City is 20.52
23th team West Brom is 21.8
24th team Wolves is 23.73
25th team Leeds United is 28.19
26th team Norwich City is 28.92
27th team Hull City is 39.4
28th team Sunderland is 43.64
29th team Everton is 44.65


# LSTM

In [4]:
def set_dataset(X, y):
    train_ds = tf.data.Dataset.from_tensor_slices((X, y))
    return train_ds

def preprocessing(df, sequences):
    vals = df.values
    teams = np.unique(df['team'].values)
    print(teams)
    X = []
    y = []
    X_test = []
    X_test_teams = []

    df_dummies = pd.get_dummies(df, columns=['team'])
    df_filtered = df_dummies.drop(columns=['year'])

    for team in teams:
        df_team = df_filtered.loc[df['team'] == team]
        #print(df_team)
        team_standings = df_team.values
        # print(f"len(team_standings) {len(team_standings)}")
        for i in range(0, len(team_standings)):
            # print(f"team {team} {i}th year")
            if i + sequences < len(team_standings):
                x_values = team_standings[i:i+sequences]
                #print(x_values)
                X.append(x_values)
                y_value = team_standings[i+sequences, 0]
                #print(y_value)
                y.append(y_value)
            elif i + sequences == len(team_standings):
                x_values = team_standings[i:i+sequences]
                X_test.append(x_values)
                print(f"team {team}")
                X_test_teams.append(team)

    X = np.array(X)
    y = np.array(y)
    X_test = np.array(X_test)
    print(X.shape)
    print(y.shape)
    print(X_test.shape)
    return X, y, X_test, X_test_teams

In [5]:
def set_lstm(n_timesteps, n_features, hidden, n_outputs):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(hidden, input_shape=(n_timesteps, n_features)))
    model.add(tf.keras.layers.Dropout(0.1))
    #model.add(Dense(hidden, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='relu'))
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse'])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.summary()
    return model

def predict(model, X_test):
    lstm_preds = model.predict(X_test)
    inds = np.argsort(lstm_preds[:,0])
    print(inds)
    print(lstm_preds.shape)

    for i, ind in enumerate(inds):
        team = X_test_teams[ind]
        pred = lstm_preds[ind, 0]
        print(f"{i+1}th team {team} is {str(np.round(pred, 1))}")

def train(n_timesteps, n_features, hidden, output_size, batch_size, epochs):
    model = set_lstm(n_timesteps, n_features, hidden, output_size)
    # Compile the model
    # optimizer = tf.keras.optimizers.Adam()
    # model.compile(optimizer=optimizer, loss='mse')

    # train_ds = set_dataset(X, y)
    print(X.shape)
    X_train, X_val = X[:100,:,:], X[100:,:]
    y_train, y_val = y[:100], y[100:]

    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2, validation_split=0.2)
    # evaluate model
    _, accuracy = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)

    print(accuracy)
    return model

In [6]:
n_timesteps = 3
n_features = 31
hidden = 64
output_size = 1  # labels are from 1-21
batch_size = 4
epochs = 100

X, y, X_test, X_test_teams = preprocessing(df, n_timesteps)

model = train(n_timesteps, n_features, hidden, output_size, batch_size, epochs)


['Arsenal' 'Aston Villa' 'Bournemouth' 'Brighton' 'Burnley' 'Cardiff City'
 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield' 'Hull City'
 'Leeds United' 'Leicester City' 'Liverpool' 'Man City' 'Man United'
 'Middlesbrough' 'Newcastle' 'Norwich City' 'Sheffield United'
 'Southampton' 'Stoke City' 'Sunderland' 'Swansea' 'Tottenham' 'Watford'
 'West Brom' 'West Ham' 'Wolves']
team Arsenal
team Aston Villa
team Bournemouth
team Brighton
team Burnley
team Cardiff City
team Chelsea
team Crystal Palace
team Everton
team Fulham
team Huddersfield
team Hull City
team Leeds United
team Leicester City
team Liverpool
team Man City
team Man United
team Newcastle
team Norwich City
team Sheffield United
team Southampton
team Stoke City
team Sunderland
team Swansea
team Tottenham
team Watford
team West Brom
team West Ham
team Wolves
(136, 3, 31)
(136,)
(29, 3, 31)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape 

Epoch 56/100
27/27 - 0s - loss: 8.3208 - mse: 8.3208 - val_loss: 27.4334 - val_mse: 27.4334 - 107ms/epoch - 4ms/step
Epoch 57/100
27/27 - 0s - loss: 7.4094 - mse: 7.4094 - val_loss: 28.5386 - val_mse: 28.5386 - 110ms/epoch - 4ms/step
Epoch 58/100
27/27 - 0s - loss: 8.5122 - mse: 8.5122 - val_loss: 27.0121 - val_mse: 27.0121 - 118ms/epoch - 4ms/step
Epoch 59/100
27/27 - 0s - loss: 8.2498 - mse: 8.2498 - val_loss: 26.9299 - val_mse: 26.9299 - 114ms/epoch - 4ms/step
Epoch 60/100
27/27 - 0s - loss: 8.3675 - mse: 8.3675 - val_loss: 28.4841 - val_mse: 28.4841 - 111ms/epoch - 4ms/step
Epoch 61/100
27/27 - 0s - loss: 7.0578 - mse: 7.0578 - val_loss: 27.5153 - val_mse: 27.5153 - 89ms/epoch - 3ms/step
Epoch 62/100
27/27 - 0s - loss: 8.3759 - mse: 8.3759 - val_loss: 28.5339 - val_mse: 28.5339 - 89ms/epoch - 3ms/step
Epoch 63/100
27/27 - 0s - loss: 7.4269 - mse: 7.4269 - val_loss: 26.7753 - val_mse: 26.7753 - 103ms/epoch - 4ms/step
Epoch 64/100
27/27 - 0s - loss: 8.5008 - mse: 8.5008 - val_loss: 3

In [7]:
predict(model, X_test)

[15 14 16  6  0 24 13 27  3 17 28  7  8  4 23 20 25  1  5 26 19 12  2 10
  9 18 21 22 11]
(29, 1)
1th team Man City is 1.1
2th team Liverpool is 1.1
3th team Man United is 1.8
4th team Chelsea is 2.9
5th team Arsenal is 7.7
6th team Tottenham is 9.2
7th team Leicester City is 9.5
8th team West Ham is 12.1
9th team Brighton is 12.5
10th team Newcastle is 13.1
11th team Wolves is 13.1
12th team Crystal Palace is 13.6
13th team Everton is 14.2
14th team Burnley is 14.9
15th team Swansea is 15.2
16th team Southampton is 15.6
17th team Watford is 16.4
18th team Aston Villa is 16.4
19th team Cardiff City is 16.6
20th team West Brom is 18.1
21th team Sheffield United is 18.5
22th team Leeds United is 18.6
23th team Bournemouth is 19.3
24th team Huddersfield is 20.2
25th team Fulham is 20.3
26th team Norwich City is 20.7
27th team Stoke City is 20.8
28th team Sunderland is 20.8
29th team Hull City is 21.0
